In [1]:
import torch
from model import load_resnet_model
import os
from train_regression_weighted_loss_loaded import train_model
import re
from dataloader import create_dataloader
from tqdm import tqdm
from dense_weight import DenseWeight

2024-02-02 09:09:04.387141: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-02 09:09:04.387185: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-02 09:09:04.387978: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-02 09:09:04.393933: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-02 09:09:05.811777: W tensorflow/compiler/tf2

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = load_resnet_model('resnet50', num_classes=1)

/opt/conda/envs/pytorch/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/envs/pytorch/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
def get_file_paths(directory):
    """
    Get all file paths in the specified directory with the specified file extension.

    Args:
    - directory (str): The directory to search for files.
    - file_extension (str): The file extension to filter by.

    Returns:
    - List[str]: A list of file paths.
    """
    file_paths = []
    for file in os.listdir(directory):
        file_path = os.path.join(directory, file)
        file_paths.append(file_path)
    return file_paths

# Use a regular expression to extract the batch number from the filename
def extract_batch_number(file_path):
    match = re.search(r"data_batch_(\d+)", file_path)
    if match:
        return int(match.group(1))
    else:
        return -1  # If for some reason a file doesn't match the pattern


In [4]:
train_files = sorted(get_file_paths('./data/records_unscaled/train/'), key= extract_batch_number)
val_files = sorted(get_file_paths('./data/records_unscaled/val/'), key= extract_batch_number)

train_loaders = [create_dataloader(file, True, 64) for file in train_files]
val_loaders = [create_dataloader(file, True, 64) for file in val_files]

In [5]:
training_labels = []
for train_loader in train_loaders:
    for _, labels in tqdm(train_loader):
        training_labels.extend(labels.detach().cpu().numpy())

dense_weight_model = DenseWeight(0.2)
dense_weight_model.fit(training_labels)

100%|██████████| 4/4 [00:00<00:00, 40.09it/s]


In [6]:
train_model(model, dense_weight_model, train_loaders, val_loaders, device, epochs=30)

Epoch 1/30, Train Loss: 0.3235, Train R2: -0.5756, Val Loss: 0.1099, Val R2: 0.4875
Model saved as checkpoint_epoch_1.pth
Epoch 2/30, Train Loss: 0.1099, Train R2: 0.4716, Val Loss: 0.1815, Val R2: 0.1640
Epoch 3/30, Train Loss: 0.0986, Train R2: 0.5251, Val Loss: 0.1209, Val R2: 0.4433
Epoch 4/30, Train Loss: 0.0906, Train R2: 0.5631, Val Loss: 0.0989, Val R2: 0.5357
Model saved as checkpoint_epoch_4.pth
Epoch 5/30, Train Loss: 0.0812, Train R2: 0.6080, Val Loss: 0.1089, Val R2: 0.4913
Epoch 6/30, Train Loss: 0.0719, Train R2: 0.6529, Val Loss: 0.1120, Val R2: 0.4727
Epoch 7/30, Train Loss: 0.0636, Train R2: 0.6923, Val Loss: 0.1126, Val R2: 0.4661
Epoch 8/30, Train Loss: 0.0628, Train R2: 0.6961, Val Loss: 0.1501, Val R2: 0.3085
Epoch 9/30, Train Loss: 0.0594, Train R2: 0.7122, Val Loss: 0.1736, Val R2: 0.1991
Epoch 10/30, Train Loss: 0.0853, Train R2: 0.5866, Val Loss: 0.1473, Val R2: 0.3245
Epoch 11/30, Train Loss: 0.0630, Train R2: 0.6953, Val Loss: 0.1398, Val R2: 0.3486
Epoch 12